# Query tests

In [1]:
[global]
from dsc.query_engine import Query_Processor as qp
import pandas as pd

def test_syntax(obj, *args):
    try:
        res = obj(*args)
        return res
    except Exception as e:
        print(e)
        assert type(e).__name__ in ['DBError', 'FormatError']
        return None

def test_outcome(obj, fn, verbose, *args):
    res = test_syntax(obj, *args)
    if res is None:
        return
    if verbose:
        for query in res.get_queries():
            print(query)
    if fn.endswith('.xlsx'):
        writer = pd.ExcelWriter(fn)
        res.output_table.to_excel(writer, 'Sheet1', index = False)
        if len(res.output_tables) > 1:
            for table in res.output_tables:
                res.output_tables[table].to_excel(writer, table, index = False)
        writer.save()
    else:
        # is csv
        res.output_table.to_csv(fn, index = False)

ash_db = 'data/dsc_result.db'
reg_db = 'data/reg_result.db'
cause_db = 'data/cause_result.db'

In [2]:
[syntax]
parameter: db = ''
parameter: t = ''
parameter: c = []
parameter: g = []
t = t.split()
res = test_syntax(qp, db, t, c, g)

In [3]:
[outcome]
parameter: db = ''
parameter: t = ''
parameter: c = []
parameter: g = []
parameter: verbose = 0
output: "/tmp/1.csv"
t = t.split()
test_outcome(qp, str(_output), verbose, db, t, c, g)

In [4]:
%sosrun default:1
[1]
# F: undefined module or group name
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist shu.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

Cannot find module ``shu`` in DSC results ``data/dsc_result.db``.


In [5]:
%sosrun default:2
[2]
# F: field name is not pipeline variable
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist simulate.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

Cannot find variable ``mse`` in module ``simulate``


In [6]:
%sosrun default:3 -s force
[3]
# F: overlapping groups
run('touch /tmp/1.csv')
sos_run("outcome", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist tmp.mse score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'],
        g = ["tmp: score_beta, score_pi0", "score: score_beta, score_pi0"])

touch /tmp/1.csv
Overlapping groups ``tmp = ['score_beta', 'score_pi0']`` and ``score = ['score_beta', 'score_pi0']`` not allowed! You should drop the one that causes the conflict, or use, eg, -g "tmp:" to erase the other one if it is build-in.


In [7]:
%sosrun default:4
[4]
# P: use built-in group names
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

In [8]:
%sosrun default:5
[5]
# F: empty table returned
sos_run("syntax", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp < 20', 'shrink.mixcompdist = "normal"'],
        g = ["score: score_beta, score_pi0"])

No results found for targets ``simulate.nsamp, shrink.mixcompdist, score.mse`` under condition ``(simulate.nsamp < 20) AND (shrink.mixcompdist = "normal")``


In [9]:
%sosrun query_show -s force
[query_show]
# P: display query.
sos_run("outcome", db = ash_db,
        t = 'simulate.nsamp shrink.mixcompdist score.mse',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'],
        verbose = 1)

SELECT simulate.DSC_REPLICATE AS simulate_DSC_FIELD_DSC_REPLICATE, simulate.nsamp AS simulate_DSC_FIELD_nsamp, shrink.mixcompdist AS shrink_DSC_FIELD_mixcompdist, score_beta.__output__ AS score_beta_DSC_VAR_mse FROM score_beta INNER JOIN shrink ON score_beta.__parent__ = shrink.__id__ INNER JOIN simulate ON shrink.__parent__ = simulate.__id__ WHERE ((simulate.nsamp > 20) AND (shrink.mixcompdist == "normal"))
SELECT simulate.DSC_REPLICATE AS simulate_DSC_FIELD_DSC_REPLICATE, simulate.nsamp AS simulate_DSC_FIELD_nsamp, shrink.mixcompdist AS shrink_DSC_FIELD_mixcompdist, score_pi0.__output__ AS score_pi0_DSC_VAR_mse FROM score_pi0 INNER JOIN shrink ON score_pi0.__parent__ = shrink.__id__ INNER JOIN simulate ON shrink.__parent__ = simulate.__id__ WHERE ((simulate.nsamp > 20) AND (shrink.mixcompdist == "normal"))


In [10]:
%sosrun default:6 -s force
[6]
# P: Handle empty field input: return the file path
sos_run("outcome", db = ash_db,
        t = 'simulate shrink score',
        c = ['simulate.nsamp > 20', 'shrink.mixcompdist = "normal"'])

In [11]:
%sosrun default:7 -s force
[7]
# P: handle group merger
sos_run("outcome", db = reg_db,
        t = 'simulate.scenario analyze score score.error')
observed = get_output('(head -n 2 /tmp/1.csv && tail -n +3 /tmp/1.csv | sort) | head').strip().split('\n')
expected = '''
DSC,simulate,simulate.scenario,analyze,analyze.output.file,score,score.error:output,score.output.file
1,dense,NA,en,en/dense_1_en_1,sq_err,sq_err/dense_1_en_1_sq_err_1,sq_err/dense_1_en_1_sq_err_1
1,dense,NA,lasso,lasso/dense_1_lasso_1,sq_err,sq_err/dense_1_lasso_1_sq_err_1,sq_err/dense_1_lasso_1_sq_err_1
1,dense,NA,ridge,ridge/dense_1_ridge_1,sq_err,sq_err/dense_1_ridge_1_sq_err_1,sq_err/dense_1_ridge_1_sq_err_1
1,en_sim,eg1,en,en/en_sim_1_en_1,sq_err,sq_err/en_sim_1_en_1_sq_err_1,sq_err/en_sim_1_en_1_sq_err_1
1,en_sim,eg1,lasso,lasso/en_sim_1_lasso_1,sq_err,sq_err/en_sim_1_lasso_1_sq_err_1,sq_err/en_sim_1_lasso_1_sq_err_1
1,en_sim,eg1,ridge,ridge/en_sim_1_ridge_1,sq_err,sq_err/en_sim_1_ridge_1_sq_err_1,sq_err/en_sim_1_ridge_1_sq_err_1
1,en_sim,eg2,en,en/en_sim_2_en_1,sq_err,sq_err/en_sim_2_en_1_sq_err_1,sq_err/en_sim_2_en_1_sq_err_1
1,en_sim,eg2,lasso,lasso/en_sim_2_lasso_1,sq_err,sq_err/en_sim_2_lasso_1_sq_err_1,sq_err/en_sim_2_lasso_1_sq_err_1
1,en_sim,eg2,ridge,ridge/en_sim_2_ridge_1,sq_err,sq_err/en_sim_2_ridge_1_sq_err_1,sq_err/en_sim_2_ridge_1_sq_err_1
'''.strip().split('\n')
assert observed == expected

In [12]:
%sosrun default:8 -s force
[8]
# P: handle group merger
sos_run("outcome", db = cause_db,
        t = 'simulate.q cis.ci_lwr cis.ci_upr summ_probs.prob cis')
observed = get_output('(head -n 2 /tmp/1.csv && tail -n +3 /tmp/1.csv | sort) | head').strip().split('\n')
expected = '''
DSC,simulate.q,cis,cis.ci_lwr:output,cis.ci_upr:output,cis.output.file,summ_probs,summ_probs.prob:output
1,0.0,gamma_ci,gamma_ci/simulate_1_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_1_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_1_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_1_cause_grid_adapt_1_gamma_lfsr_1
1,0.1,gamma_ci,gamma_ci/simulate_2_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_2_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_2_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_2_cause_grid_adapt_1_gamma_lfsr_1
1,0.2,gamma_ci,gamma_ci/simulate_3_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_3_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_3_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_3_cause_grid_adapt_1_gamma_lfsr_1
1,0.5,gamma_ci,gamma_ci/simulate_4_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_4_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_4_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_4_cause_grid_adapt_1_gamma_lfsr_1
1,0.7,gamma_ci,gamma_ci/simulate_5_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_5_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_5_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_5_cause_grid_adapt_1_gamma_lfsr_1
1,1.0,gamma_ci,gamma_ci/simulate_6_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_6_cause_grid_adapt_1_gamma_ci_1,gamma_ci/simulate_6_cause_grid_adapt_1_gamma_ci_1,gamma_lfsr,gamma_lfsr/simulate_6_cause_grid_adapt_1_gamma_lfsr_1
2,0.0,gamma_ci,gamma_ci/simulate_7_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_7_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_7_cause_grid_adapt_4_gamma_ci_2,gamma_lfsr,gamma_lfsr/simulate_7_cause_grid_adapt_4_gamma_lfsr_2
2,0.1,gamma_ci,gamma_ci/simulate_8_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_8_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_8_cause_grid_adapt_4_gamma_ci_2,gamma_lfsr,gamma_lfsr/simulate_8_cause_grid_adapt_4_gamma_lfsr_2
2,0.2,gamma_ci,gamma_ci/simulate_9_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_9_cause_grid_adapt_4_gamma_ci_2,gamma_ci/simulate_9_cause_grid_adapt_4_gamma_ci_2,gamma_lfsr,gamma_lfsr/simulate_9_cause_grid_adapt_4_gamma_lfsr_2
'''.strip().split('\n')
assert observed == expected